In [1]:
import xml.etree.cElementTree as ET
import os
import nltk
import string
import random
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
#import re
import numpy as np
from bert.tokenization import FullTokenizer
#from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from keras.utils import np_utils

W0405 22:06:05.088888 16128 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4718195028315360765
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10193742398
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6725706880284593926
physical_device_desc: "device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


In [3]:
# Initialize session
sess = tf.Session()

In [4]:
# BERT parameters and tokenizer
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

# Preprocess training data

## Get files for parsing

In [5]:
wd = os.path.dirname(os.path.abspath('__file__'))

folders = ['\\data_all\\06_training-RiskFactors-Complete-Set1\\','\\data_all\\09_training-RiskFactors-Complete-Set2\\']
filenames = []

for folder in folders:
    for file in os.listdir(str(wd)+folder):
        filename=os.fsdecode(os.fsencode((str(wd)+folder+file)))
        if filename.endswith(('.xml')):
            filenames.append(filename)

## Define functions to create training tokens and labels

In [6]:
def create_tokenizer_from_hub_module():
    bert_module = hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
    [
        tokenization_info["vocab_file"],
        tokenization_info["do_lower_case"],
    ])
    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [7]:
# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0405 22:06:10.070299 16128 deprecation.py:323] From C:\Users\Sarah\Anaconda3\envs\Tensorflow-GPU-Keras\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0405 22:06:13.824102 16128 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [8]:
def spans(tokenizer, text):
    tokens = nltk.word_tokenize(text.lower())
    offset = 0
    start_indices = []
    end_indices = []
    for token in tokens:
        offset = text.find(token, offset)
        start, end = offset, offset+len(token)
        offset += len(token)
        start_indices.append(start)
        end_indices.append(end)
    return tokens, start_indices, end_indices

In [9]:
def create_training_input(tokenizer, file_path):
    tree = ET.ElementTree(file=file_path)
    root = tree.getroot()
    
    text = root.find('TEXT').text.lower()
    
    tokens, start, end = spans(tokenizer, text)
    
    labels_list = []
    label_start = []
    label_end = []

    for item in root.find("TAGS"):
        if item.tag == 'PHI':
            pass
        elif item.tag == 'SMOKER':
            label = "I-" + (item.tag + "." + item.attrib['status']).lower().replace(" ", "_")
        elif item.tag == 'FAMILY_HIST':
            label = "I-" + (item.tag + "." + item.attrib['indicator']).lower().replace(" ", "_")
        elif item.tag == 'MEDICATION':
            label = "I-" + (item.tag + "." + item.attrib['type1'] + "." + item.attrib['type2'] + "." + item.attrib['time']).lower().replace(" ", "_")
        else:
            label = "I-" + (item.tag + "." + item.attrib['indicator'] + "." + item.attrib['time']).lower().replace(" ", "_")

        for sub_item in item.findall(item.tag):
            if 'start' in sub_item.attrib.keys():
                labels_list.append(label)
                label_start.append(int(sub_item.attrib['start']))
                label_end.append(int(sub_item.attrib['end']))
                
    io_labels = []

    count = 0
    while len(start) > count:
        if start[count] in label_start:
            label_start_index = label_start.index(start[count])
            end_index = label_end[label_start_index]
            word_label = labels_list[label_start_index]
            phrase = text[start[count]:end_index]
            phrase_tokens = nltk.word_tokenize(phrase)
            for word in phrase_tokens:
                io_labels.append(word_label)
                count += 1
        else:
            io_labels.append("O")
            count += 1  
    
    tokens_bert = []
    io_labels_bert = []
    for token, label in zip(tokens, io_labels):
        new_tokens = tokenizer.tokenize(token)
        for token in new_tokens:
            tokens_bert.append(token)
            io_labels_bert.append(label)
    tokens_bert.insert(0, '[CLS]')
    tokens_bert.append('[SEP]')
    io_labels_bert.insert(0, "O")
    io_labels_bert.append("O")
    
    input_ids = tokenizer.convert_tokens_to_ids(tokens_bert)
    return tokens_bert, io_labels_bert, input_ids

## Get training data from .xml files

In [10]:
train_tokens = []
train_labels = []
train_input_ids = []

split_index = int(len(filenames)*0.75)
random.seed(42)
random.shuffle(filenames)
for file in filenames[:split_index]:
    tokens, io_labels, input_ids = create_training_input(tokenizer, file)

    train_tokens.extend(tokens)
    train_labels.extend(io_labels)
    train_input_ids.extend(input_ids)

In [11]:
dev_tokens = []
dev_labels = []
dev_input_ids = []

for file in filenames[split_index:]:
    tokens, io_labels, input_ids = create_training_input(tokenizer, file)
    
    dev_tokens.extend(tokens)
    dev_labels.extend(io_labels)
    dev_input_ids.extend(input_ids)

# Create BERT compatible features and labels

In [12]:
label_to_int = {}
int_to_label = {}

unique_labels = list(set(dev_labels+train_labels))
for i in range(len(unique_labels)):
    label_to_int[unique_labels[i]] = i
    int_to_label[i] = unique_labels[i]

In [13]:
def convert_label_to_int(labels, dictionary):
    train_classes = []
    for label in labels:
        train_classes.append(dictionary[label])
    return train_classes

In [14]:
train_classes = convert_label_to_int(train_labels, label_to_int)
dev_classes = convert_label_to_int(dev_labels, label_to_int)

In [15]:
# train_input_masks = np.array([1] * len(train_classes))
# dev_input_masks = np.array([1] * len(dev_classes))

# train_segment_ids = np.array([0] * len(train_classes))
# dev_segment_ids = np.array([0] * len(train_classes))

# train_labels = np_utils.to_categorical(train_classes)
# dev_labels = np_utils.to_categorical(dev_classes)

# train_input_ids = np.array(train_input_ids)
# dev_input_ids = np.array(dev_input_ids)

In [16]:
def create_seqs(classes, ids, dictionary, max_seq_length=100):
    input_ids, input_masks, segment_ids, labels = [], [], [], []

    start = 0
    end = max_seq_length
    while start < len(classes):
        while (classes[start:end][-1] != dictionary["O"]):
            end -= 1
            if end == start:
                end += max_seq_length
                break
        seq_segment_ids = [0] * max_seq_length
        seq_classes = classes[start:end] + [dictionary["O"]] * (max_seq_length-len(classes[start:end]))
        seq_ids = ids[start:end] + [0] * (max_seq_length-len(ids[start:end]))
        seq_mask = [1] * len(ids[start:end]) + [0] * (max_seq_length-len(ids[start:end]))

        input_ids.append(seq_ids)
        input_masks.append(seq_mask)
        segment_ids.append(seq_segment_ids)
        labels.append(seq_classes)

        start = end
        end += max_seq_length
        
    return (
        np.array(input_ids), 
        np.array(input_masks), 
        np.array(segment_ids), 
        #np.array(labels)
        #np.array(labels).reshape(-1,1),
        np_utils.to_categorical(np.array(labels)),
    )

In [17]:
max_seq_length = 100
# Create features
(train_input_ids, train_input_masks, train_segment_ids, train_labels) = create_seqs(train_classes, train_input_ids, label_to_int, max_seq_length)
(dev_input_ids, dev_input_masks, dev_segment_ids, dev_labels) = create_seqs(dev_classes, dev_input_ids, label_to_int, max_seq_length)

In [18]:
print(train_input_ids.shape, train_labels.shape)

(6475, 100) (6475, 100, 93)


# Build BERT model

In [19]:
class BertLayer(tf.layers.Layer):
    def __init__(self, n_fine_tune_layers=10, **kwargs):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            bert_path,
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )

        trainable_vars = self.bert.variables

        # Remove unused layers
        trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]

        # Select how many layers to fine tune
        trainable_vars = trainable_vars[-self.n_fine_tune_layers :]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)
            
        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
            "pooled_output"
        ]
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [25]:
class BERTBase:
    @staticmethod
    def build(max_seq_length):
        in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
        in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
        in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
        bert_inputs = [in_id, in_mask, in_segment]

        bert_output = BertLayer(n_fine_tune_layers=5)(bert_inputs)
        return bert_output, bert_inputs
    

In [51]:
class DenseHead:
    @staticmethod
    def build(bert_base):
        headModel = bert_base
        headModel = tf.keras.layers.Flatten(name="flatten")(headModel)
        headModel = tf.keras.layers.Dense(9300, activation="relu")(headModel)
        headModel = tf.keras.layers.Reshape((-1,93))(headModel)
        headModel = tf.keras.layers.Dense(93, activation="softmax")(headModel)
        
        return headModel
        

In [52]:
def cat_acc(y_true, y_pred):
    return tf.keras.metrics.categorical_accuracy(K.flatten(y_true), K.flatten(y_pred))

In [53]:
bert_output, bert_inputs = BERTBase.build(max_seq_length)
dense_head = DenseHead.build(bert_output)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=dense_head)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[cat_acc])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0405 22:17:57.175815 16128 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0405 22:18:00.563734 16128 saver.py:1483] Saver not created because there are no variables in the graph to restore


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 100)          0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
bert_layer_8 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

In [54]:
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)
    
initialize_vars(sess)

model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([dev_input_ids, dev_input_masks, dev_segment_ids], dev_labels),
    epochs=3,
    batch_size=50
)

Train on 6475 samples, validate on 2243 samples
6475/6475 [==============================] - 71s 11ms/sample - loss: 0.4862 - cat_acc: 0.0000e+00 - val_loss: 0.3817 - val_cat_acc: 0.0000e+00


In [57]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

preds = model.predict([dev_input_ids, dev_input_masks, dev_segment_ids])

def convert_pred_to_label(preds):
    out = []
    for pred in preds:
        out_i = []
        for p in pred:
            p_i = np.argmax(p)

In [74]:
for i in preds:
    for j in i:
        label = int_to_label[np.argmax(j)]
        if label != 'O':
            print(int_to_label[np.argmax(j)])

In [71]:
int_to_label[9]

'O'

In [ ]:
label_to_int = {}
int_to_label = {}

def convert_label_to_int(labels, dictionary):
    train_classes = []
    for label in labels:
        train_classes.append(dictionary[label])
    return train_classes


4768/4768 [==============================] - 64s 13ms/step
idx2tag = {i: w for w, i in tags2index.items()}
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PADword", "O"))
        out.append(out_i)
    return out

    
pred_labels = pred2label(test_pred)
test_labels = test2label(y_te[:149*32])
print(classification_report(test_labels, pred_labels))
               precision   recall  f1-score   support

        org       0.69      0.66      0.68      2061
        tim       0.88      0.84      0.86      2148
        gpe       0.95      0.93      0.94      1591
        per       0.75      0.80      0.77      1677
        geo       0.85      0.89      0.87      3720
        art       0.23      0.14      0.18        49
        eve       0.33      0.33      0.33        33
        nat       0.47      0.36      0.41        22

avg / total       0.82      0.82      0.82     11301

# SCRATCH

In [ ]:
train_labels[0]

In [ ]:
# Build modelhttp://localhost:8888/notebooks/Final%20Project/Single%20Model.ipynb#SCRATCH
def build_model(max_seq_length): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=3)(bert_inputs)
    flatten = tf.keras.layers.Flatten(name="flatten")(bert_output)
    dense = tf.keras.layers.Dense(256, activation='relu')(flatten)
    pred = tf.keras.layers.Dense(93, activation='softmax')(dense) # number of prediction nodes should equal number of unique labels
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [ ]:
model = build_model(max_seq_length)

# Instantiate variables
initialize_vars(sess)

model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([dev_input_ids, dev_input_masks, dev_segment_ids], dev_labels),
    epochs=1,
    batch_size=32
)